In [40]:
import json
import pandas as pd
import time
from tqdm import tqdm
import jieba
import re
import gensim
from gensim.models import word2vec

In [3]:
infodf = pd.read_csv('infoall.csv').drop('Unnamed: 0.1',axis = 1).set_index('Unnamed: 0')

In [4]:
#  加载停用词列表
def load_stopword():
    f_stop = open('hit_stopwords.txt', encoding='utf-8')  # 自己的中文停用词表
    sw = [line.strip() for line in f_stop]  # strip() 方法用于移除字符串头尾指定的字符（默认为空格）
    f_stop.close()
    return sw

# 中文分词并且去停用词
def seg_word(sentence):
    file_userDict = 'hit_stopwords.txt'  # 自定义的词典
    jieba.load_userdict(file_userDict)

    sentence_seged = jieba.cut(sentence.strip(),cut_all=False)
    stopwords = load_stopword()
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '/t':
                outstr += word
                outstr += " "
    return outstr

In [37]:
f = open('sentences.txt','w')
for stocknum , stockinfo in tqdm(infodf.iterrows(),total=infodf.shape[0]):
    f.write(str(stockinfo['name']).strip('nan') + ' ')
    f.write(str(stockinfo['shortname']).strip('nan')+ ' ')
    f.write(str(stockinfo['prename']).strip('nan')+ ' ')
    f.write(seg_word(str(stockinfo['operation'])))
    f.write(seg_word(str(stockinfo['operation_scope'])))
    f.write(seg_word(str(stockinfo['introduction'])))
    f.write(str(stockinfo['legal_representative']).strip('nan')+ ' ')
    f.write(str(stockinfo['actual_controller']).strip('nan')+ ' ')
    f.write(str(stockinfo['office']).strip('nan')+ ' ')
    f.write(str(stockinfo.name).strip('nan'))
    f.write('\n')
f.close()

100%|██████████| 5432/5432 [02:13<00:00, 40.60it/s]


In [44]:
sentences = word2vec.Text8Corpus("sentences.txt")
model=gensim.models.Word2Vec(sentences,sg=1,window=10,min_count=2,negative=3,sample=0.001,hs=1,workers=4)
#sg=1是skip—gram算法，对低频词敏感，默认sg=0为CBOW算法
#size是神经网络层数，值太大则会耗内存并使算法计算变慢，一般值取为100到200之间。
#window是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b个词，后面看b个词（b在0-3之间随机）
#min_count是对词进行过滤，频率小于min-count的单词则会被忽视，默认值为5。
#negative和sample可根据训练结果进行微调，sample表示更高频率的词被随机下采样到所设置的阈值，默认值为1e-3,
#negative: 如果>0,则会采用negativesamping，用于设置多少个noise words
#hs=1表示层级softmax将会被使用，默认hs=0且negative不为0，则负采样将会被选择使用。
#workers是线程数，此参数只有在安装了Cpython后才有效，否则只能使用单核

In [62]:
model.wv.wmdistance('','复华')

1.2823679386417206